# Why DCGAN instead of GAN?

Generative Adversarial Networks (GANs) use two networks to generate data (generator) and classify if it is real or generated (discriminator). These two networks continuously tries to improve their abilities and after training one can supply some noise to the generator and it will generate some output, e.g. a cat if it has been trained on cat pictures.  

Problems with GANs are that they are hard to train, the model can collapse, slight changes in variables/architecture can destabilize the model, i.e. training GANs is somewhat of an art according to some. Deep Convolutional GANs (DCGANs) solves/mitigates these problems.  

### Main differences between DCGANs and GANs
Making three changes to GANs allows them to become more stable and produce better results.  

The first change is to replace pooling functions with convolution layers.  
The second change is to remove fully connected layers on top of convolutional layers.  
The third change to use batch normalization (although with some caveats...)

![title](./guidelines.png)

Source: DCGAN-paper by Radford, Metz & Chintala  
https://arxiv.org/pdf/1511.06434.pdf

## General Architecture of DCGANs
The general architecture of a DCGAN is very similar to a GAN, both can even contain deep convolutional layers despite the names. The architecture consists of a generator which generates an image out of some initial noise and a discriminator which classifies real and fake images. The picture below gives an overview of a GAN/DCGAN.

![title](./GAN-architecture.png)

Source: https://towardsdatascience.com/deep-generative-models-25ab2821afd3 

### DCGAN loss function
Our DCGAN will use two cost functions, one for the generator and one for the discriminator. One can choose among several different cost functions for the the generator, we chose a heuristically motivated cost function which ensures that the "losing" network's gradient is strong. 

- z = Input distribution to generator 
- G(z) = Output from the generator given z
- D(G(z)) = The discriminators classification of G(z), i.e. real or false

**Generator loss function**     
![title](./generatorCost.png)


**Discriminator loss function**
![title](./discriminatorCost.png)

Source: NIPS 2016 Tutorial: Generative Adversarial Networks  
https://arxiv.org/pdf/1701.00160.pdf

### Initial hyperparameters  
The DCGAN-paper presents some hyperparameters which worked well

- Mini-batch size: 128
- Weights initialization: Zero-centered normal distribution with standard deviation 0.02
- LeakyReLU leak: 0.2
- Optimizer: Adam with learning rate 0.0002 and Beta1 0.5 (default, 0.9, caused oscillations). Beta1 is momentum decay


In [2]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# to make this notebook's output stable across runs (from lab2)
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 1.11.0


In [5]:
class DCGAN(object):
    
    def __init__(self):
        """
        Constructs DCCGAN with tf.Keras

        """
        
        # Set parameters
        self.rows = 28
        self.columns = 28
        self.channels = 1
        self.shape = (self.rows, self.columns, self.channels) # Image shape 
        self.batch_size = 128
        
        # Build the models
        self.generator = self._buildGenerator()
        self.discriminator = self._buildDiscriminator()
    
        # Compile the models
        self.generator.compile()
        self.discriminator.compile()
        
        # Train the models
        
        
    # Currently following this: https://julianzaidi.wordpress.com/2017/04/24/deep-convolution-gan-dcgan-architecture-and-training/
    def _buildGenerator(self):
        """
        Constructs the generator part of the DCGAN
        """
        ##########
        # Layers #
        ##########
        # 1. Input
        # 2. Reshape layer
        # 3. Transposed conv layer
        # 4. Transposed conv layer
        # 5. Transposed conv layer
        # 6. Transposed conv layer
        # 7. Transposed conv layer
        
        #model = tf.keras.Input() # Sequential or Functional API?
        # In the blog post they use a reshape, is that neccessary? Investigate
        noise = np.random.normal(loc=0 , scale=0.2 , shape=(self.batch_size, self.rows, 1, 1))
        model = tf.keras.Sequential()
        
        model.add(layers.Conv2DTranspose(input_shape=(100,), kernel_size=(7,7), 
                                         strides=(1,1), 
                                         kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02)))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation("relu"))
        
        model.add(layers.Conv2DTranspose())
        model.add(layers.BatchNormalization())
        model.add(layers.Activation("relu"))
        
        model.add(layers.Conv2DTranspose())
        model.add(layers.BatchNormalization())
        model.add(layers.Activation("relu"))
        
        model.add(layers.Conv2DTranspose(activation='tanh'))
    
        model.summary()
        
        return model
    
    def _buildDiscriminator(self):
        """
        Constructs the discriminator part of the DCGAN
        """
        ##########
        # Layers #
        ##########
        # 1. Input
        # 2. Conv layer
        # 3. Conv layer
        # 4. Conv layer
        # 5. Conv layer
        # 6. Dense layer
        
        model = tf.keras.Sequential()
        
        model.add(layers.Conv2D(input_shape=(None, 28, 28, 1), kernel_size=(), strides=()))
        model.add(layers.LeakyReLU(alpha=0.2))
        
        model.add(layers.Conv2D())
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU(alpha=0.2))
        
        model.add(layers.Conv2D())
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU(alpha=0.2))
        
        model.add(layers.Conv2D())
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU(alpha=0.2))
        
        model.add(layers.Conv2D())
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU(alpha=0.2))
        
        model.add(layers.Flatten())
        model.add(layers.Dense(activation=tf.nn.sigmoid))
    
        model.summary()
        
        return model
    
    def trainModel(self):
        """
        Trains the generator and discriminator
        """
        


In [11]:
#noise = np.random.normal(loc=0 , scale=0.2 , size=(3, 2, 1, 1))
#print(noise)

[[[[ 0.13897881]]

  [[ 0.11966697]]]


 [[[-0.47545968]]

  [[ 0.2212227 ]]]


 [[[ 0.19881932]]

  [[-0.03660763]]]]
